## 데이터

In [106]:
# 셀레니움 잘 깔렸는지 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.10.18
4.33.0


In [107]:
import os
import requests
import pandas as pd
import sqlite3
import re
import sys

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller  # setup chrome options

import urllib.request
from urllib.request import urlretrieve
import time

import warnings
warnings.filterwarnings('ignore')

In [108]:
market_df = pd.read_csv('./markets_data/markets_cleaned.csv', encoding='utf-8-sig')

In [109]:
market_df.head(1)

,시장ID,시장명,도로명주소,시도,시군구,아케이드 보유 여부,엘리베이터_에스컬레이터_보유여부,고객지원센터 보유 여부,시장전용 고객주차장_보유여부,종합콜센터_보유여부,...,유아놀이방_보유여부,수유센터_보유여부,물품보관함_보유여부,자전거보관함_보유여부,외국인 안내센터_보유여부,시장유형,시장개설주기,점포수,공중화장실 보유여부,사용가능상품권
0,경기도 가평군 설악눈메골시장,설악눈메골시장,경기도 가평군 설악면 신천중앙로 104-1,경기도,가평군,N,N,N,Y,N,...,N,N,N,Y,N,NaN,NaN,NaN,NaN,NaN


# 프로세스

In [5]:
 # chrome_path = "/content/drive/MyDrive/1_public/chromedriver" # chromedirver 경로 알려주기

# 네이버 지도 리뷰

In [110]:
market_df = market_df.rename(columns={
    '시장ID': 'mID',
})

In [111]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
import re

#지역명이 포함된 식당명을 변수로 지정
items = market_df['mID']
print(items)

#검색할 식당 데이터와 url을 담을 데이터 프레임 생성
df = pd.DataFrame(columns=['mID', 'naverURL'])
df = pd.read_csv('./markets_data/markets_review_url.csv', encoding='utf-8-sig')

#데이터 프레임이 잘 만들어졌는지 확인
df['mID'] = items
df

# 식당 url 얻기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('--disable-images')  # 이미지 로딩 비활성화
options.add_argument('--disable-javascript')  # 불필요한 JS 비활성화
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
options.add_argument("--no-sandbox")              # 샌드박스 비활성화
options.add_argument("--disable-dev-shm-usage")   # /dev/shm 사용 방지
options.add_argument("--disable-gpu")             # GPU 비활성화
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 5)   # 최대 5초까지 기다림

# User-Agent 설정으로 봇 감지 회피
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

0                    경기도 가평군 설악눈메골시장
1                    경기도 가평군 가평잣고을시장
2                     경기도 가평군 청평여울시장
3                       경기도 고양시 원당시장
4                       경기도 고양시 능곡시장
                    ...             
1383                제주특별자치도 제주시 신산시장
1384              제주특별자치도 제주시 함덕오일시장
1385    제주특별자치도 제주시 한림민속오일시장(한림오일시장)
1386              제주특별자치도 제주시 한림매일시장
1387              제주특별자치도 제주시 화북종합시장
Name: mID, Length: 1388, dtype: object


In [112]:
# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

In [113]:
# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [44]:
for i, keyword in enumerate(df['mID'].tolist()):
    print(f"[{i+1}] {keyword} 검색 중...")

    try:

        # 검색 url 만들기
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
    
        driver.get(naver_map_search_url)
        
        # css를 찾을때 까지 10초 대기
        time_wait(10, 'div.input_box > input.input_search')
        
        # (1) 검색창 찾기
        search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
        search.send_keys(keyword)  # 검색어 입력
        search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
        time.sleep(2) 
        
        # 검색 프레임 변경
        driver.switch_to.frame("searchIframe")
        time.sleep(1) 
    
        # 시장 정보가 있다면 첫번째 식당의 url을 가져오기
        if len(driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li')) != 0:   
            # 시장 정보 클릭        
    
            driver.execute_script('return document.querySelector(".place_bluelink").click()')
            time.sleep(2)
            
            # 검색한 플레이스의 개별 페이지 저장
            tmp = driver.current_url
            print(f"tmp: {tmp}")
            res_code = re.findall(r"place/(\d+)", tmp)
            print(f"res_code: {res_code}")
            final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' 
            print(final_url)
            df['naverURL'][i]=final_url 
        
    except: 
        df['naverURL'][i]= ''
        print('none') 
    
driver.close()

#식당명과 url이 잘 얻어져왔는지 확인하기
print(df)

#url을 얻어오지 못한 식당 확인
print(df.loc[df['naverURL']==''])
#결측치로 입력된 식당 확인
print(df.loc[df['naverURL'].isna()])
## 식당 리뷰 url이 들어가있지 않은 경우 직접 검색하여 데이터에 넣어주기!!! ##

[1] 경기도 가평군 설악눈메골시장 검색 중...
tmp: https://map.naver.com/p/search/%EA%B2%BD%EA%B8%B0%EB%8F%84%20%EA%B0%80%ED%8F%89%EA%B5%B0%20%EC%84%A4%EC%95%85%EB%88%88%EB%A9%94%EA%B3%A8%EC%8B%9C%EC%9E%A5/place/1418775378?c=15.00,0,0,0,dh&placePath=/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202506151345&locale=ko&svcName=map_pcv5&searchText=%EA%B2%BD%EA%B8%B0%EB%8F%84%20%EA%B0%80%ED%8F%89%EA%B5%B0%20%EC%84%A4%EC%95%85%EB%88%88%EB%A9%94%EA%B3%A8%EC%8B%9C%EC%9E%A5
res_code: ['1418775378']
https://pcmap.place.naver.com/restaurant/1418775378/review/visitor#
[2] 경기도 가평군 가평잣고을시장 검색 중...
tmp: https://map.naver.com/p/search/%EA%B2%BD%EA%B8%B0%EB%8F%84%20%EA%B0%80%ED%8F%89%EA%B5%B0%20%EA%B0%80%ED%8F%89%EC%9E%A3%EA%B3%A0%EC%9D%84%EC%8B%9C%EC%9E%A5/place/38738070?c=15.00,0,0,0,dh&placePath=/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202506151346&locale=ko&svcName=map_pcv5&searchText=%EA%B2%BD%EA%B8%B0%EB%8F%84%20%EA%B0%80%ED%8F%89%EA%B5%B0%20%EA%B0%80%ED%8F%89%EC%9E%A3%EA%B3%A0%EC%9D%84%EC%8B%9C%EC%9

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x0xec3783+63299]
	GetHandleVerifier [0x0xec37c4+63364]
	(No symbol) [0x0xcf0f70]
	(No symbol) [0x0xd2c9a8]
	(No symbol) [0x0xd5e436]
	(No symbol) [0x0xd59f05]
	(No symbol) [0x0xd59486]
	(No symbol) [0x0xcc3a05]
	(No symbol) [0x0xcc3f5e]
	(No symbol) [0x0xcc43ed]
	GetHandleVerifier [0x0x111e073+2531379]
	GetHandleVerifier [0x0x1119372+2511666]
	GetHandleVerifier [0x0xee9efa+220858]
	GetHandleVerifier [0x0xeda548+156936]
	GetHandleVerifier [0x0xee0c7d+183357]
	(No symbol) [0x0xcc36d0]
	(No symbol) [0x0xcc2edd]
	GetHandleVerifier [0x0x1257f1c+3817180]
	BaseThreadInitThunk [0x0x76825d49+25]
	RtlInitializeExceptionChain [0x0x777bd09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x777bd021+561]


## 비어있는 url 재입력

In [18]:
# nan인 행의 인덱스만 추출
nan_idx = df[df['naverURL'].isna()].index.tolist()
print(f"URL이 비어있는 키워드 수: {len(nan_idx)}\n")

for i, idx in enumerate(nan_idx, start=1):
    keyword = df.at[idx, 'mID']
    print(f"[{i}/{len(nan_idx)}] {keyword} 재검색 중...")

    try:
        # 검색 URL로 이동
        driver.get(f'https://map.naver.com/v5/search/{keyword}/place')

        # 검색창 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.input_box > input.input_search'))
        )

        # 검색창 입력 및 검색
        search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
        search.clear()
        search.send_keys(keyword)
        search.send_keys(Keys.ENTER)
        time.sleep(2)

        # 검색 결과 iframe으로 전환
        driver.switch_to.frame("searchIframe")
        time.sleep(1)

        # 검색 결과가 있다면
        results = driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li')
        if results:
            # 첫 번째 결과 클릭
            driver.execute_script('document.querySelector(".place_bluelink").click()')
            time.sleep(2)

            # 프레임 복귀
            driver.switch_to.default_content()
            time.sleep(1)

            # 현재 URL에서 place ID 추출
            current_url = driver.current_url
            print("tmp:", current_url)
            res_code = re.findall(r"place/(\d+)", current_url)

            if res_code:
                final_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#'
                print("✓ 최종 URL:", final_url)
                df.at[idx, 'naverURL'] = final_url
            else:
                print("⚠ place ID 추출 실패")
                df.at[idx, 'naverURL'] = ''
        else:
            print("⚠ 검색 결과 없음")
            df.at[idx, 'naverURL'] = ''

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        df.at[idx, 'naverURL'] = ''

    finally:
        # 혹시 프레임이 안 돌아왔으면 복귀
        try:
            driver.switch_to.default_content()
        except:
            pass

# 드라이버 종료
driver.quit()

# 결과 확인
print("\n=== 최종 결과 ===")
print(df[['mID', 'naverURL']])
print(f"\n성공: {df['naverURL'].astype(bool).sum()}개")
print(f"실패: {(~df['naverURL'].astype(bool)).sum()}개")

URL이 비어있는 키워드 수: 92

[1/92] 경기도 성남시 미금현대벤처빌시장 재검색 중...
⚠ 검색 결과 없음
[2/92] 경기도 성남시 모란가축시장 재검색 중...
⚠ 검색 결과 없음
[3/92] 경기도 여주시 여주한글시장(여주중앙로상점가) 재검색 중...
⚠ 검색 결과 없음
[4/92] 경기도 용인시 죽전로데오상점가시장 재검색 중...
⚠ 검색 결과 없음
[5/92] 경기도 화성시 궁평항해오름수산시장 재검색 중...
⚠ 검색 결과 없음
[6/92] 서울특별시 강북구 백년시장(강북종합전통시장) 재검색 중...
⚠ 검색 결과 없음
[7/92] 서울특별시 강북로 수유중앙시장 재검색 중...
⚠ 검색 결과 없음
[8/92] 서울특별시 강서구 등마루시장 재검색 중...
⚠ 검색 결과 없음
[9/92] 서울특별시 광진구 노룬산시장(건물형) 재검색 중...
⚠ 검색 결과 없음
[10/92] 서울특별시 광진구 조양시장(건물형) 재검색 중...
⚠ 검색 결과 없음
[11/92] 서울특별시 광진구 신성종합시장(건물형) 재검색 중...
⚠ 검색 결과 없음
[12/92] 서울특별시 광진구 중곡제일시장(이오상사) 재검색 중...
⚠ 검색 결과 없음
[13/92] 서울특별시 동대문구 청량종합도매시장(깡통시장) 재검색 중...
⚠ 검색 결과 없음
[14/92] 서울특별시 성북구 돌곶이시장(구 석관황금시장) 재검색 중...
⚠ 검색 결과 없음
[15/92] 서울특별시 양천구 목동깨비시장 (목3동시장) 재검색 중...
⚠ 검색 결과 없음
[16/92] 서울특별시 종로구 동대문종합시장 신관(동대문쇼핑타운) 재검색 중...
⚠ 검색 결과 없음
[17/92] 인천광역시 강화군 강화중앙시장(a동b동) 재검색 중...
⚠ 검색 결과 없음
[18/92] 대전광역시 중구 산성뿌리전통시장 재검색 중...
⚠ 검색 결과 없음
[19/92] 충청남도 보령시 대천항종합수산물시장 재검색 중...
⚠ 검색 결과 없음
[20/92] 충청남도 천안시 천안중앙시장(구 남산중앙시장) 재검색 중...
⚠ 검색 

In [33]:
# naver URL 직접 넣기
df.loc[df['mID'] == '경상남도 창녕군 영산시장', 'naverURL'] = 'https://pcmap.place.naver.com/restaurant/13348192/review/visitor#'

In [34]:
# 비어있는 행 삭제
df = df[~(df['naverURL'].isna() | (df['naverURL'] == ''))]

In [35]:
# 개수 확인
df['naverURL'].isna().sum()

np.int64(0)

In [78]:
# csv 파일로도 저장하여 url이 빈 값이 있는지 반드시 확인할 것 !
# url이 없으면 코드 실행이 중단되므로 반드시 url 데이터를 확인할 것!!
df.to_csv('./markets_data/markets_review_url.csv', encoding='utf-8-sig', index=False)

# 네이버 지도 리뷰 크롤링

In [125]:
url_df = pd.read_csv('./markets_data/markets_review_url.csv', encoding='utf-8-sig')
review_df = pd.read_csv('./markets_data/markets_reviews_temp.csv', encoding='utf-8-sig')

# URL 테이블에서 필요한 컬럼만 가져오기
url_df = url_df[['mID', 'naverURL']]
review_df = review_df[['mID', 'rate', 'review']]

# # 재수집 대상 시장 (특정 시장 + 누락 시장)
# retry_mids = [
#     "서울특별시 송파구 새마을시장",
#     "서울특별시 양천구 목사랑시장",
#     "세종특별자치시 금남면 세종대평시장",
    
#     "서울특별시 관악구 봉천현대시장",
#     "경기도 성남시 은행시장",
#     "경기도 부천시 중동사랑시장(부천중동시장)",
#     "경기도 시흥시 시흥삼미시장",
#     "경기도 안양시 안양호계종합시장",
#     "서울시 동대문구 청량리전통시장",
#     "서울특별시 동작구상도로102 성대시장"
# ]

# # 삭제 기준 목록: retry_mids에 포함된 mID 제거
# review_df = review_df[~review_df['mID'].isin(retry_mids)].reset_index(drop=True)

# 리뷰가 없는 시장 mID 추출
done_mids = set(review_df['mID'].unique())
all_mids = set(url_df['mID'].unique())
not_done_mids = all_mids - done_mids # 리뷰가 없는 시장

# 총 재시도 대상
# target_mids = set(retry_mids) | not_done_mids
target_mids = not_done_mids

# 최종 대상 DataFrame 추출
df = url_df[url_df['mID'].isin(target_mids)].reset_index(drop=True)

In [126]:
df

,mID,naverURL
0,경상북도 울진군 후포시장,https://pcmap.place.naver.com/restaurant/13347...
1,경상북도 의성군 금성시장,https://pcmap.place.naver.com/restaurant/15895...
2,경상북도 의성군 봉양시장,https://pcmap.place.naver.com/restaurant/15895...
3,경상북도 의성군 의성공설재래시장,https://pcmap.place.naver.com/restaurant/16898...
4,경상북도 의성군 의성염매시장,https://pcmap.place.naver.com/restaurant/13347...
...,...,...
544,제주특별자치도 제주시 신산시장,https://pcmap.place.naver.com/restaurant/17092...
545,제주특별자치도 제주시 함덕오일시장,https://pcmap.place.naver.com/restaurant/13345...
546,제주특별자치도 제주시 한림민속오일시장(한림오일시장),https://pcmap.place.naver.com/restaurant/13224...
547,제주특별자치도 제주시 한림매일시장,https://pcmap.place.naver.com/restaurant/13347...


In [104]:
df

,mID,naverURL
0,경기도 부천시 중동사랑시장(부천중동시장),https://pcmap.place.naver.com/restaurant/16312...
1,경기도 성남시 은행시장,https://pcmap.place.naver.com/restaurant/18394...
2,경기도 시흥시 시흥삼미시장,https://pcmap.place.naver.com/restaurant/13304...
3,경기도 안양시 안양호계종합시장,https://pcmap.place.naver.com/restaurant/35150...
4,서울특별시 관악구 봉천현대시장,https://pcmap.place.naver.com/restaurant/13306...
...,...,...
1044,제주특별자치도 제주시 신산시장,https://pcmap.place.naver.com/restaurant/17092...
1045,제주특별자치도 제주시 함덕오일시장,https://pcmap.place.naver.com/restaurant/13345...
1046,제주특별자치도 제주시 한림민속오일시장(한림오일시장),https://pcmap.place.naver.com/restaurant/13224...
1047,제주특별자치도 제주시 한림매일시장,https://pcmap.place.naver.com/restaurant/13347...


In [127]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()

# User-Agent 설정으로 봇 감지 회피
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

count = 0 #
current = 0 #현재 진행 상황

goal = len(df['mID']) # 총 시장 수

# 데이터 프레임으로 만들 빈 리스트 생성
rev_list = []

for i in range(len(df)): 
    
    current += 1
    print('진행상황 : ', current,'/', goal,sep="")
    mID = df.at[i, 'mID']
    url = df.at[i, 'naverURL']
    print(f"\n[{current}/{goal}] 시장명: {mID}")

    
    # 시장 리뷰 개별 url 접속
    driver.get(df['naverURL'][i]) 
    thisurl = df['naverURL'][i]
    time.sleep(2)
    print('현재 수집중인 시장 : ', df['mID'][i])

    # 리뷰 더보기 버튼 누르기
    while True: 
        try:
            review_items = driver.find_elements(By.CSS_SELECTOR, "#app-root .place_section.k1QQ5 .place_section_content ul > li")
            print(f"  📄 현재 리뷰 수: {len(review_items)}")

            if len(review_items) >= 200:
                print("  ⛔ 리뷰 200개 도달 → 더보기 중단")
                break

            more_btn = driver.find_element(By.CSS_SELECTOR,
                "#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k1QQ5 > div.NSTUp > div > a.fvwqf")
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(1)
            more_btn.click()
            time.sleep(2)
                        
        except NoSuchElementException:
            print("-모든 리뷰 더보기 완료-")
            break    
        
    # 리뷰 저장
    if len(review_items) == 0:
        rev_list.append([mID, rating, ''])
        print("  ⚠ 리뷰 없음")
        
    else:
        for j, review in enumerate(review_items[:200]):
            try:
                # "더보기" 클릭 시도
                try:
                    more_btn = review.find_element(By.CSS_SELECTOR, "div.pui__vn15t2 > a.pui__wFzIYl")
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", more_btn)
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pui__vn15t2 > a.pui__wFzIYl")))
                    try:
                        more_btn.click()
                    except ElementClickInterceptedException:
                        driver.execute_script("arguments[0].click();", more_btn)
                    time.sleep(0.3)
                except NoSuchElementException:
                    pass

                # 리뷰 텍스트 추출
                text_elem = review.find_element(By.CSS_SELECTOR, "div.pui__vn15t2 > a:nth-child(1)")
                review_text = text_elem.text.strip()
                rev_list.append([mID, '', review_text])
                print(f"    ✔ 리뷰 {j+1}: {review_text[:30]}...")

            except NoSuchElementException:
                print("    ⚠ 리뷰 텍스트 없음")
                continue

    # 10개 단위 저장
    if current % 10 == 0:
        df2 = pd.DataFrame(rev_list, columns=["mID", "rate", "review"])
        df2.to_csv(
            "./markets_data/markets_reviews_temp.csv",
            mode='a',
            index=False,
            encoding="utf-8-sig",
            escapechar='\\'
        )
        print(f"✅ 저장 완료 (현재까지 {current}개)")

driver.close()

#스크래핑한 데이터를 데이터 프레임으로 만들기  
column = ["mID", 'rate', "review"]
df2 = pd.DataFrame(rev_list, columns=column)
df2 

진행상황 : 1/549

[1/549] 시장명: 경상북도 울진군 후포시장
현재 수집중인 시장 :  경상북도 울진군 후포시장
  📄 현재 리뷰 수: 10
  📄 현재 리뷰 수: 10
  📄 현재 리뷰 수: 11
-모든 리뷰 더보기 완료-
    ✔ 리뷰 1: 불친절하네요...
    ✔ 리뷰 2: 굿...
    ✔ 리뷰 3: 친절하세요...
    ✔ 리뷰 4: 조아요...
    ✔ 리뷰 5: 좋아요...
    ✔ 리뷰 6: 사장님께서 엄청 친절하셔요

6월엔 킹크랩 먹었고
8월...
    ✔ 리뷰 7: 조아요...
    ✔ 리뷰 8: 좋아요...
    ✔ 리뷰 9: 굿입니다...
    ✔ 리뷰 10: 대게가 저렴해요 ㅎㅎㅎ...
    ✔ 리뷰 11: 굿...
진행상황 : 2/549

[2/549] 시장명: 경상북도 의성군 금성시장
현재 수집중인 시장 :  경상북도 의성군 금성시장
  📄 현재 리뷰 수: 5
-모든 리뷰 더보기 완료-
    ✔ 리뷰 1: 쪼아요.쯔아...
    ✔ 리뷰 2: 굿입니다...
    ✔ 리뷰 3: 친절해요...
    ✔ 리뷰 4: 상품 다양하고 좋아요...
    ✔ 리뷰 5: 굿입니다...
진행상황 : 3/549

[3/549] 시장명: 경상북도 의성군 봉양시장
현재 수집중인 시장 :  경상북도 의성군 봉양시장
  📄 현재 리뷰 수: 4
-모든 리뷰 더보기 완료-
    ✔ 리뷰 1: 굿...
    ✔ 리뷰 2: 굿...
    ✔ 리뷰 3: 냠냠...
    ✔ 리뷰 4: 육회맛있네요...
진행상황 : 4/549

[4/549] 시장명: 경상북도 의성군 의성공설재래시장
현재 수집중인 시장 :  경상북도 의성군 의성공설재래시장
  📄 현재 리뷰 수: 5
-모든 리뷰 더보기 완료-
    ✔ 리뷰 1: 다양한볼거리가많아요...
    ✔ 리뷰 2: 굿...
    ✔ 리뷰 3: 닭발유명한시장...
    ✔ 리뷰 4: 굿...
    ✔ 리뷰 5: 굿...
진행상황 : 5/549

[5/549] 시장명: 경상북도

,mID,rate,review
0,경상북도 울진군 후포시장,,불친절하네요
1,경상북도 울진군 후포시장,,굿
2,경상북도 울진군 후포시장,,친절하세요
3,경상북도 울진군 후포시장,,조아요
4,경상북도 울진군 후포시장,,좋아요
...,...,...,...
13036,제주특별자치도 제주시 화북종합시장,,ㅈㅇㅇ
13037,제주특별자치도 제주시 화북종합시장,,굿
13038,제주특별자치도 제주시 화북종합시장,,맛있어요
13039,제주특별자치도 제주시 화북종합시장,,들기름 고소하고 맛나요


# 후처리

In [130]:
market_df = pd.read_csv('./markets_data/markets_reviews_temp.csv', encoding='utf-8-sig')
market_df

,mID,rate,review
0,경기도 가평군 설악눈메골시장,NaN,NaN
1,경기도 가평군 설악눈메골시장,NaN,매장에 쉽게 이용할 수 있었어
2,경기도 가평군 가평잣고을시장,NaN,NaN
3,경기도 가평군 가평잣고을시장,NaN,맛있어요. 친절해요
4,경기도 가평군 가평잣고을시장,NaN,좋아요
...,...,...,...
643239,제주특별자치도 제주시 제주시동문재래시장,NaN,귤향과즐 맛있음!
643240,제주특별자치도 제주시 제주시동문재래시장,NaN,좋아요
643241,제주특별자치도 제주시 보성시장,NaN,보성시장 통큰순대집 국밥 냄새안나고 신선하고 엄청 푸짐해요 반찬들도 전부 맛있어요\...
643242,제주특별자치도 제주시 보성시장,NaN,좋아요


In [131]:
# mID, rate, review 컬럼 모두 동일한 경우 중복 제거
market_df = market_df.drop_duplicates(subset=['mID', 'review']).reset_index(drop=True)
market_df.shape

(21125, 3)

In [132]:
market_df['mID'].nunique()

1289

In [133]:
market_df.to_csv('./markets_data/markets_reviews_temp.csv', encoding='utf-8-sig', index=False)

# 카카오맵 리뷰

In [ ]:
# import pandas as pd
# import time
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options
# from selenium.common.exceptions import NoSuchElementException, TimeoutException
# from tqdm import tqdm
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# def extract_review():
#     try:
#         html = driver.page_source
#         soup = BeautifulSoup(html, 'html.parser')
#         rev = []
#         # 리뷰 목록에서 텍스트 추출 - 정확한 선택자 사용
#         review_list = soup.select('#mainContent > div.main_detail > div.detail_cont > div.section_comm.section_review > div.group_review > ul > li')
#         print(f"  - 발견된 리뷰 수: {len(review_list)}")

#         for li in review_list:
#             try:
#                 # 정확한 리뷰 텍스트 선택자 사용
#                 review_text = li.select_one('div > div.area_review > div > div.review_detail > div.wrap_review > a > p')
#                 if review_text:
#                     text = review_text.get_text(strip=True)
#                     if text:
#                         rev.append(text)
#             except Exception as e:
#                 print(f"  - 개별 리뷰 추출 중 오류: {e}")
#                 continue

#         return rev
#     except Exception as e:
#         print(f"  - 리뷰 추출 중 오류 발생: {e}")
#         return []

# search_keywords = df['시장ID'].tolist()
# market_names = df['시장ID'].tolist()  # 시장명 리스트는 결과 저장용으로 유지

# # 결과 저장용 딕셔너리
# rev_dict = {}

# # 크롬 옵션
# options = Options()
# options.add_argument('--start-maximized')
# # options.add_argument('--headless')  # 필요시 주석 해제

# driver = webdriver.Chrome(options=options)
# wait = WebDriverWait(driver, 10)  # 최대 10초 대기

# for market in tqdm(search_keywords):
#     rev_dict[market] = {'별점': '', '리뷰': []}  # for문 시작할 때 미리 초기화
#     print(f'\n[{market}] 검색 중...')

#     try:
#         # 카카오맵 접속
#         driver.get('https://map.kakao.com/')
#         time.sleep(3)

#         # 검색창 찾기 및 검색어 입력
#         search_box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#search\.keyword\.query')))
#         search_box.clear()
#         search_box.send_keys(market)
#         search_box.send_keys(Keys.ENTER)
#         time.sleep(3)

#         # 첫 번째 검색 결과 클릭 - 별점/리뷰 버튼이 있는지 확인
#         try:
#             first_result = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#info\.search\.place\.list > li.PlaceItem.clickArea.PlaceItem-ACTIVE > div.rating.clickArea > span.score > a')))
#             first_result.click()
#             time.sleep(3)

#             # 새로 열린 페이지로 전환
#             driver.switch_to.window(driver.window_handles[-1])
#             time.sleep(3)

#             # 별점 추출
#             try:
#                 rating_elem = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#mainContent > div.main_detail > div.detail_cont > div.section_comm.section_review > div.group_total > div.head_total > span > span.num_star')))
#                 rating = rating_elem.text.strip()
#                 print(f"  - 별점: {rating}")
#             except Exception as e:
#                 print(f"  - 별점 추출 실패: {e}")
#                 rating = ''

#             # 리뷰 추출
#             rev = extract_review()
#             rev_dict[market] = {'별점': rating, '리뷰': rev}
#             print(f"  - 저장된 리뷰 수: {len(rev)}")

#             # 새로 열린 창 닫고 원래 창으로 돌아가기
#             driver.close()
#             driver.switch_to.window(driver.window_handles[0])

#         except Exception as e:
#             print(f"  - 별점/리뷰 버튼이 없거나 클릭할 수 없음: {e}")
#             rev_dict[market] = {'별점': '', '리뷰': []}

#         # 중간 저장 (성공/실패 모두)
#         result_df = pd.DataFrame([
#             {'시장ID': k, '별점': v['별점'], '리뷰': v['리뷰']} for k, v in rev_dict.items()
#         ])
#         result_df.to_csv('review_result_kakao.csv', index=False, encoding='utf-8-sig')
#         print(f"  - 중간 저장 완료")

#     except Exception as e:
#         print(f"[{market}] 처리 중 오류 발생: {e}")
#         # 오류 발생 시에도 창 전환 처리
#         if len(driver.window_handles) > 1:
#             driver.close()
#             driver.switch_to.window(driver.window_handles[0])
#         continue

# # 최종 결과 저장
# print("\n최종 결과 저장 중...")
# result_df = pd.DataFrame([
#     {'시장ID': k, '별점': v['별점'], '리뷰': v['리뷰']} for k, v in rev_dict.items()
# ])
# result_df.to_csv('review_result_kakao.csv', index=False, encoding='utf-8-sig')
# print("크롤링 완료!")

In [ ]:
driver.quit()

In [ ]:
print((df['naverURL'] == '').sum())    # 빈 문자열 개수

# 리뷰 데이터 병합

In [ ]:
# # 리뷰 데이터 병합
# df[['평점', '리뷰수']] = df['시장ID'].apply(lambda x: pd.Series(get_review_data(x)))

In [ ]:
# # 저장
# conn = sqlite3.connect('market_review.db')
# market_df.to_sql('market_info', conn, if_exists='replace', index=False)
# conn.close()